# ٍSurvival Analysis

## Table of Contents: 
### Preface
### A. K-means Clustering
### B. Survival Analysis 

## Preface

## 1. Import necessary libraries:

In [222]:
import os # for the directory 
import csv # reading files
import pandas as pd  # data analysis tools
import numpy as np
import re
from pandas import DataFrame # easier to analysis
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
fname=r"C:\Users\mohab\gdc_download_PhiSci" # locate the location 
os.chdir(r"C:\Users\mohab\gdc_download_PhiSci") # assign the default work folder 
list_names = os.listdir(fname) # to include all files into one list variable

## 2. Import the transfered file produced earlier using log2 transformations:

In [223]:
# Read the corresponding CSV file we need to select genes from
finalTable = pd.read_csv('Final_table_Silhouette_col.csv')

## 3. Explore Dataframe file

In [224]:
print(type(finalTable))

<class 'pandas.core.frame.DataFrame'>


In [225]:
finalTable.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000071889,ENSG00000171867,ENSG00000178209,ENSG00000103266,ENSG00000105968,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063
0,TCGA054244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,0.115143,0.947719,-0.666491,-0.079637,-0.190571,-0.853176,1.066205,1.054552,-0.385172,-0.662107
1,TCGA054249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,-2.441444,0.319628,-0.563266,-0.621155,-0.907976,1.212749,0.511445,0.707088,1.079129,0.000597
2,TCGA054250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,-1.032840,0.998781,0.103940,0.008959,-0.118967,-1.726892,1.242704,-0.037235,-0.687928,-1.010352
3,TCGA054382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,0.574486,0.451952,-0.258683,-0.114534,-0.615276,-0.553858,0.493738,-0.247060,-1.593753,0.013734
4,TCGA054384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,0.323376,-0.595971,0.455942,0.061927,-1.115623,-0.471615,-0.261625,0.136249,-1.401781,0.168867


In [226]:
finalTable.shape

(512, 5431)

## A. K-means Clustering 

### A.1. Applying plain Kmeans to the dataframe

In [227]:
# Find Kmeans for all z-scored normalization
cols = list(finalTable.columns)
cols.remove('Patients')
finalTable[cols].head()

,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,ENSG00000179833,...,ENSG00000071889,ENSG00000171867,ENSG00000178209,ENSG00000103266,ENSG00000105968,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063
0,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,1.193198,...,0.115143,0.947719,-0.666491,-0.079637,-0.190571,-0.853176,1.066205,1.054552,-0.385172,-0.662107
1,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,-0.113202,...,-2.441444,0.319628,-0.563266,-0.621155,-0.907976,1.212749,0.511445,0.707088,1.079129,0.000597
2,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,-0.081538,...,-1.032840,0.998781,0.103940,0.008959,-0.118967,-1.726892,1.242704,-0.037235,-0.687928,-1.010352
3,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,0.236620,...,0.574486,0.451952,-0.258683,-0.114534,-0.615276,-0.553858,0.493738,-0.247060,-1.593753,0.013734
4,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,-0.432786,...,0.323376,-0.595971,0.455942,0.061927,-1.115623,-0.471615,-0.261625,0.136249,-1.401781,0.168867


In [228]:
# Find Kmeans for all z-scored normalization
kmeans = KMeans(n_clusters=3)
kmeans.fit(finalTable[cols].values)
y_kmeans = kmeans.predict(finalTable[cols])
y_kmeans

array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 2, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 1, 1,
       2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 2, 0, 0, 0, 2,
       1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 0, 1, 2, 1, 1,
       1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2,
       0, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 2, 0,
       0, 0, 1, 0, 1, 2, 2, 0, 2, 0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1,

In [229]:
# add the column of clusters to the dataframe finalTable
finalTable['Cluster'] = y_kmeans

In [230]:
finalTable.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000171867,ENSG00000178209,ENSG00000103266,ENSG00000105968,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,Cluster
0,TCGA054244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,0.947719,-0.666491,-0.079637,-0.190571,-0.853176,1.066205,1.054552,-0.385172,-0.662107,1
1,TCGA054249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.319628,-0.563266,-0.621155,-0.907976,1.212749,0.511445,0.707088,1.079129,0.000597,1
2,TCGA054250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,0.998781,0.103940,0.008959,-0.118967,-1.726892,1.242704,-0.037235,-0.687928,-1.010352,1
3,TCGA054382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,0.451952,-0.258683,-0.114534,-0.615276,-0.553858,0.493738,-0.247060,-1.593753,0.013734,1
4,TCGA054384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,-0.595971,0.455942,0.061927,-1.115623,-0.471615,-0.261625,0.136249,-1.401781,0.168867,1


## A.2 Applying One-Hot encoder

In [231]:
# Get one hot encoding of columns 'Cluster', # Drop column as it is now encoded
#finalTable = finalTable.drop('Cluster',axis = 1)
finalTable_hotted = pd.get_dummies(finalTable, columns=['Cluster'], prefix='', prefix_sep='')

In [232]:
# split patient ID into a list, exclude the last digits
finalTable_hotted['Patients']=finalTable_hotted['Patients'].apply(lambda x: '-'.join([x[:4], x[4:6], x[6:]]))

In [233]:
finalTable_hotted.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000103266,ENSG00000105968,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,-0.079637,-0.190571,-0.853176,1.066205,1.054552,-0.385172,-0.662107,0,1,0
1,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,-0.621155,-0.907976,1.212749,0.511445,0.707088,1.079129,0.000597,0,1,0
2,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,0.008959,-0.118967,-1.726892,1.242704,-0.037235,-0.687928,-1.010352,0,1,0
3,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,-0.114534,-0.615276,-0.553858,0.493738,-0.247060,-1.593753,0.013734,0,1,0
4,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,0.061927,-1.115623,-0.471615,-0.261625,0.136249,-1.401781,0.168867,0,1,0


In [234]:
#to convert the output Data Frame Table to a correponding CSV format
finalTable_hotted.to_csv(r'Final_table_one_hotted.csv', index = False)

In [236]:
finalTable_hotted.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000103266,ENSG00000105968,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,-0.079637,-0.190571,-0.853176,1.066205,1.054552,-0.385172,-0.662107,0,1,0
1,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,-0.621155,-0.907976,1.212749,0.511445,0.707088,1.079129,0.000597,0,1,0
2,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,0.008959,-0.118967,-1.726892,1.242704,-0.037235,-0.687928,-1.010352,0,1,0
3,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,-0.114534,-0.615276,-0.553858,0.493738,-0.247060,-1.593753,0.013734,0,1,0
4,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,0.061927,-1.115623,-0.471615,-0.261625,0.136249,-1.401781,0.168867,0,1,0


### B. Applying Survival Analysis

In [237]:
finalTable_survival = pd.read_csv(r"clinical_survival_analysis.csv")

In [238]:
# extract all columns 
cols = list(finalTable_survival.columns)

In [239]:
# fetch all respective columns ['case_submitter_id', 'vital_status', 'days_to_death', 'days_to_last_follow_up']
finalTable_survival = finalTable_survival[['case_submitter_id', 'vital_status', 'days_to_death', 'days_to_last_follow_up']]
#rename the 'case_submitter_id' to 'Patients'
finalTable_survival.rename(columns={'case_submitter_id': 'Patients', }, inplace = True)
finalTable_survival.head()

,Patients,vital_status,days_to_death,days_to_last_follow_up
0,TCGA-05-4244,Alive,'--,0
1,TCGA-05-4244,Alive,'--,0
2,TCGA-75-6203,Alive,'--,'--
3,TCGA-75-6203,Alive,'--,'--
4,TCGA-44-2665,Alive,'--,1301


In [241]:
# merege both dataframes to one dataframe
finalTable_hotted_survival = pd.merge(finalTable_hotted, finalTable_survival, on = 'Patients')
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,vital_status,days_to_death,days_to_last_follow_up
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.066205,1.054552,-0.385172,-0.662107,0,1,0,Alive,'--,0
1,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.066205,1.054552,-0.385172,-0.662107,0,1,0,Alive,'--,0
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.511445,0.707088,1.079129,0.000597,0,1,0,Alive,'--,1523
3,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.511445,0.707088,1.079129,0.000597,0,1,0,Alive,'--,1523
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,1.242704,-0.037235,-0.687928,-1.010352,0,1,0,Dead,121,'--


In [242]:
# new table consist of replicated genes
finalTable_hotted_survival.shape

(1024, 5437)

In [243]:
# sorting by Patients ID 
finalTable_hotted_survival.sort_values("Patients", inplace = True) 
# dropping ALL duplicte values 
finalTable_hotted_survival = finalTable_hotted_survival.drop_duplicates(subset='Patients', keep='first')
# rename 'vital_status'to 'Event_Observed'
finalTable_hotted_survival.rename(columns={'vital_status': 'Event_Observed', }, inplace = True)
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,Event_Observed,days_to_death,days_to_last_follow_up
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.066205,1.054552,-0.385172,-0.662107,0,1,0,Alive,'--,0
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.511445,0.707088,1.079129,0.000597,0,1,0,Alive,'--,1523
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,1.242704,-0.037235,-0.687928,-1.010352,0,1,0,Dead,121,'--
6,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,0.493738,-0.247060,-1.593753,0.013734,0,1,0,Alive,'--,607
8,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,-0.261625,0.136249,-1.401781,0.168867,0,1,0,Alive,'--,426


In [244]:
#the new shape of processed data
finalTable_hotted_survival.shape

(512, 5437)

### B.2. Changing the Alive| Dead values to True|False respectively

In [245]:
finalTable_hotted_survival['Event_Observed'] = finalTable_hotted_survival['Event_Observed'].replace(['Alive'],'False')
finalTable_hotted_survival['Event_Observed'] = finalTable_hotted_survival['Event_Observed'].replace(['Dead'],'True')
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,Event_Observed,days_to_death,days_to_last_follow_up
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.066205,1.054552,-0.385172,-0.662107,0,1,0,False,'--,0
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.511445,0.707088,1.079129,0.000597,0,1,0,False,'--,1523
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,1.242704,-0.037235,-0.687928,-1.010352,0,1,0,True,121,'--
6,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,0.493738,-0.247060,-1.593753,0.013734,0,1,0,False,'--,607
8,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,-0.261625,0.136249,-1.401781,0.168867,0,1,0,False,'--,426


### B.3. Add new column "time to event" based on 1. time to death for dead cases (TRUE) 2. time to last follow up for alive cases(FALSE)

In [247]:
finalTable_hotted_survival['Time_to_event'] = np.where((finalTable_hotted_survival.Event_Observed == True), finalTable_hotted_survival.days_to_death, finalTable_hotted_survival.days_to_last_follow_up)
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,Event_Observed,days_to_death,days_to_last_follow_up,Time_to_event
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.054552,-0.385172,-0.662107,0,1,0,False,'--,0,0
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.707088,1.079129,0.000597,0,1,0,False,'--,1523,1523
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,-0.037235,-0.687928,-1.010352,0,1,0,True,121,'--,'--
6,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,-0.247060,-1.593753,0.013734,0,1,0,False,'--,607,607
8,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,0.136249,-1.401781,0.168867,0,1,0,False,'--,426,426


In [248]:
# Remove column name 'days_to_death', 'days_to_last_follow_up'
finalTable_hotted_survival = finalTable_hotted_survival.drop(['days_to_death', 'days_to_last_follow_up'], axis = 1) 
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000099991,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,Event_Observed,Time_to_event
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,-0.853176,1.066205,1.054552,-0.385172,-0.662107,0,1,0,False,0
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,1.212749,0.511445,0.707088,1.079129,0.000597,0,1,0,False,1523
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,-1.726892,1.242704,-0.037235,-0.687928,-1.010352,0,1,0,True,'--
6,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,-0.553858,0.493738,-0.247060,-1.593753,0.013734,0,1,0,False,607
8,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,-0.471615,-0.261625,0.136249,-1.401781,0.168867,0,1,0,False,426


In [251]:
# declare an annual variable
const = 365
# convert all value to numeric
finalTable_hotted_survival['Time_to_event'] = pd.to_numeric(finalTable_hotted_survival['Time_to_event'], errors='coerce')
# divide all numbers by the const variable
#finalTable_hotted_survival['New_Time_to_event'] = finalTable_hotted_survival['Time_to_event']/const #one way
finalTable_hotted_survival['Time_to_event'] = finalTable_hotted_survival['Time_to_event'].apply(lambda x: np.true_divide(x, const)) # second way
finalTable_hotted_survival.head()

,Patients,ENSG00000146083,ENSG00000198242,ENSG00000134108,ENSG00000167700,ENSG00000070087,ENSG00000179262,ENSG00000064601,ENSG00000008517,ENSG00000070081,...,ENSG00000102265,ENSG00000133313,ENSG00000066044,ENSG00000105063,0,1,2,Event_Observed,Time_to_event,New_Time_to_event
0,TCGA-05-4244,0.589114,1.003665,-0.048534,-1.013633,-0.499692,-0.087478,-0.391675,0.877901,-0.136574,...,1.066205,1.054552,-0.385172,-0.662107,0,1,0,False,0.000000,0.000000
2,TCGA-05-4249,0.336812,0.584443,-0.868243,-1.302287,-0.770021,-1.043345,-0.803281,-0.830437,-0.350080,...,0.511445,0.707088,1.079129,0.000597,0,1,0,False,4.172603,4.172603
4,TCGA-05-4250,-0.095578,2.070690,-0.080474,-0.750315,0.943487,0.065604,-0.236112,1.444375,-2.121779,...,1.242704,-0.037235,-0.687928,-1.010352,0,1,0,True,NaN,NaN
6,TCGA-05-4382,-1.025727,0.070446,-0.155905,-1.582172,1.073434,-0.816411,0.138105,0.496662,-0.685795,...,0.493738,-0.247060,-1.593753,0.013734,0,1,0,False,1.663014,1.663014
8,TCGA-05-4384,1.654415,0.033384,1.336560,0.382178,0.165434,-1.015725,-0.028534,-1.206722,-1.645633,...,-0.261625,0.136249,-1.401781,0.168867,0,1,0,False,1.167123,1.167123


In [250]:
#to convert the output Data Frame Table to a correponding CSV format
finalTable_hotted_survival.to_csv(r'finalTable_hotted_survival_dataset.csv', index = False)